<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_2_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    986      0 --:--:-- --:--:-- --:--:--   982
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [53]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,4,Discoloration of skin [blotchy] on the neck
2,7,goiter
3,8,arthritis
4,8,arthritis


In [54]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-2.test.csv",header=None)
test.head()

,0,1
0,2,right heel pain
1,4,black and blue marks
2,8,arthritice
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [59]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [63]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.933278,5.820560,0.141239,02:01


In [64]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.763584,5.279888,0.181194,02:01


In [65]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.394058,5.233446,0.186496,02:02


In [66]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.339518,5.186959,0.190848,02:01


In [0]:
learn.save_encoder('healthnews_fold2_first')

In [68]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold2_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.667125,4.204607,0.352243,01:08


In [69]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.995348,3.922927,0.361478,01:22


In [70]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.628566,3.698769,0.410290,03:14


In [71]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.428254,3.504401,0.427441,03:17
1,3.208827,3.432787,0.432718,03:14


In [72]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.096406,3.329197,0.445910,03:22
1,2.952914,3.302069,0.444591,03:18


In [73]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.838617,3.247697,0.455145,03:22
1,2.804158,3.193714,0.459103,03:21


In [74]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.806324,3.164312,0.465699,03:24
1,2.598489,3.068470,0.468338,03:17


In [75]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.682776,3.079146,0.478892,03:08
1,2.649273,3.054466,0.481530,03:26


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.579718,3.079203,0.480211,03:11
1,2.595369,3.098237,0.481530,03:20
2,2.476645,3.071206,0.480211,03:26
3,2.420622,3.041842,0.484169,03:21


In [77]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.388919,3.011949,0.496042,03:15
1,2.446929,3.070354,0.456464,03:22
2,2.324729,3.057743,0.507916,03:26
3,2.326592,3.010936,0.507916,03:13


In [78]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.274451,3.033316,0.498681,03:09
1,2.375262,3.058402,0.494723,03:12
2,2.317841,3.072691,0.503958,03:11
3,2.272418,3.100682,0.503958,03:05


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.162165,3.045749,0.474934,03:30
1,2.377520,3.150833,0.501319,03:20
2,2.190364,3.126078,0.509235,03:23
3,2.131717,3.086714,0.507916,03:19


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.245875,3.195469,0.510554,03:12
1,2.253320,3.163489,0.469657,03:24
2,2.182687,3.076811,0.518470,03:16
3,2.146799,3.124561,0.513193,03:17


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.188964,3.175826,0.506596,03:11
1,2.210314,3.201111,0.500000,03:26
2,2.167877,3.206682,0.506596,03:12
3,2.083323,3.164159,0.506596,03:09


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.106916,3.151783,0.505277,03:30
1,2.202033,3.232754,0.515831,03:17
2,2.146343,3.179955,0.515831,03:20
3,2.142719,3.145610,0.515831,03:22


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.147008,3.244502,0.470976,03:34
1,2.204935,3.191855,0.500000,03:23
2,2.202873,3.248837,0.510554,03:37
3,2.143031,3.252606,0.515831,03:30


In [84]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.082688,3.182199,0.510554,03:15
1,2.119409,3.287626,0.502639,03:31
2,2.100623,3.166782,0.517150,03:23
3,2.020818,3.193093,0.515831,03:15


In [0]:
# save the best model

learn.save_encoder('ask-2-tuning-healthnews')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [87]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 783, 1026, 269, 269, 269, 1026, 778, 49, 16, 810, 225, 30, 320, 414, 414, 414, 228, 1035, 439, 37, 696, 37, 37, 37, 37, 775, 289, 289, 289, 289, 623, 702, 256, 49, 228, 49, 49, 301, 228, 228, 228, 228, 228, 228, 135, 140, 164, 228, 228, 228, 434, 265, 60, 228, 228, 228, 735, 245, 182, 364, 227, 607, 805, 94, 289, 289, 228, 228, 228, 228, 427, 289, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 172, 172, 228, 228, 228, 228, 228, 220, 912, 80, 80, 80, 80, 80, 318, 228, 268, 228, 92, 92, 961, 164, 164, 49, 826, 826, 228, 228, 228, 228, 289, 228, 228, 1035, 98, 265, 265, 265, 265, 108, 660, 660, 289, 638, 111, 111, 111, 100, 289, 111, 337, 120, 1026, 228, 228, 228, 228, 228, 125, 125, 125, 125, 125, 125, 125, 125, 125, 513, 289, 131, 264, 745, 228, 228, 137, 139, 142, 143, 228, 228, 847, 228, 155, 157, 1035, 537, 228, 228, 228, 604, 462, 462, 289, 289, 210, 626, 326, 735, 326, 182, 462, 462, 758, 758, 1035, 187, 187, 187

In [88]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

216
0.24942263279445728
